In [64]:
import sys
from pathlib import Path

# Add analysis tools to path
NOTEBOOK_DIR = Path.cwd()
ANALYSIS_ROOT = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == "notebooks" else NOTEBOOK_DIR
if str(ANALYSIS_ROOT) not in sys.path:
    sys.path.insert(0, str(ANALYSIS_ROOT))

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tools import wandb_io, config, paths
from tools.aggregations import runs_history_to_frame

## Configuration

In [65]:
# Sweep config - add multiple sweep IDs here
SWEEP_IDS = [
    "0hio89k0",
     "wnmohh5v"   # Primary sweep
    # "another_sweep_id",  # Add more sweep IDs here
]
ENTITY = "thomasevers9"
PROJECT = "tdmpc2-tdmpc2"

# Plot config
MAX_STEP = 1000_000  # This sweep runs for 100k steps
METRIC_KEY = "eval/episode_reward"

# Task name mapping: sweep task name -> baseline CSV name (SimbaV2 naming convention)
TASK_NAME_MAP = {
    "ball_in_cup-catch": "cup-catch",
    "finger-turn_easy": "finger-turn-easy",
    "finger-turn_hard": "finger-turn-hard",
    "cartpole-balance_sparse": "cartpole-balance-sparse",
    "cartpole-swingup_sparse": "cartpole-swingup-sparse",
    # Humanoid bench tasks
    "humanoid_h1-slide-v0": "h1-slide-v0",
    "humanoid_h1-walk-v0": "h1-walk-v0",
    "humanoid_h1-balance_simple-v0": "h1-balance-simple-v0",
    "humanoid_h1-balance_hard-v0": "h1-balance-hard-v0",
    "humanoid_h1-run-v0": "h1-run-v0",
    "humanoid_h1-stand-v0": "h1-stand-v0",
}

# BMPC uses different naming for humanoidbench (h1hand-* instead of h1-*)
BMPC_TASK_MAP = {
    "h1-slide-v0": "h1hand-slide-v0",
    "h1-walk-v0": "h1hand-walk-v0",
    "h1-balance-simple-v0": "h1hand-balance_simple-v0",
    "h1-balance-hard-v0": "h1hand-balance_hard-v0",
    "h1-run-v0": "h1hand-run-v0",
    "h1-stand-v0": "h1hand-stand-v0",
}

# Baseline roots
BASELINE_ROOTS = {
    "TDMPC2": paths.BASELINE_TDMPC2,
    "DreamerV3": paths.BASELINE_DREAMERV3,
    "SAC": paths.BASELINE_SAC,
    "EZ2": paths.PROJECT_ROOT / "results" / "ez2_parsed",
    "SimbaV2": paths.PROJECT_ROOT / "results" / "simbav2_parsed",
    "BMPC": paths.PROJECT_ROOT / "results" / "BMPC_parsed" / "dmcontrol",
    "BMPC_humanoid": paths.PROJECT_ROOT / "results" / "BMPC_parsed" / "humanoidbench",
}

# Colors for methods
COLORS = {
    "Ours": "#1f77b4",      # Blue
    "TDMPC2": "#2ca02c",    # Green
    "DreamerV3": "#ff7f0e", # Orange
    "EZ2": "#d62728",       # Red
    "SimbaV2": "#9467bd",   # Purple
    "SAC": "#8c564b",       # Brown
    "BMPC": "#17becf",      # Cyan
}

## Load Our Results from W&B

In [66]:
# Fetch sweep runs from W&B (with caching) - supports multiple sweeps
all_runs = []
for sweep_id in SWEEP_IDS:
    runs, manifest, source = wandb_io.fetch_sweep_runs(
        entity=ENTITY,
        project=PROJECT,
        sweep_id=sweep_id,
        history_keys=[METRIC_KEY, "_step"],
        use_cache=True,
        force_refresh=False,
    )
    print(f"Sweep {sweep_id}: {len(runs)} runs from {source} (fetched: {manifest.get('fetched_at', 'N/A')})")
    all_runs.extend(runs)

print(f"\nTotal: {len(all_runs)} runs from {len(SWEEP_IDS)} sweep(s)")

Sweep 0hio89k0: 5 runs from cache (fetched: 2026-01-29T11:33:29.738506+00:00)


Sweep wnmohh5v: 5 runs from remote (fetched: 2026-01-29T12:04:17.037524+00:00)

Total: 10 runs from 2 sweep(s)


In [67]:
# Convert to DataFrame
ours_df = runs_history_to_frame(
    all_runs,
    metric_key=METRIC_KEY,
    step_keys=["_step"],
    config_to_columns={"task": "task", "seed": "seed"},
)

# Normalize task names to match baseline naming
ours_df["task"] = ours_df["task"].replace(TASK_NAME_MAP)

# Rename metric column to 'reward' for consistency
ours_df = ours_df.rename(columns={METRIC_KEY: "reward"})

print(f"Our results: {len(ours_df)} rows")
print(f"Tasks: {sorted(ours_df['task'].unique().tolist())}")
print(f"Seeds: {sorted(ours_df['seed'].unique().tolist())}")
print(f"Max step: {ours_df['step'].max()}")
ours_df.head()

Our results: 98 rows
Tasks: ['dog-run', 'dog-walk', 'h1-slide-v0', 'h1-walk-v0', 'humanoid-walk']
Seeds: [1]
Max step: 600000


,task,seed,run_id,step,reward
0,dog-run,1,9y283uk7,40000,113.061874
1,dog-run,1,9y283uk7,80000,159.115860
2,dog-run,1,9y283uk7,120000,200.753708
3,dog-run,1,9y283uk7,160000,232.283600
4,dog-run,1,9y283uk7,200000,269.457947


## Load Baseline Results

In [68]:
def load_baseline_for_task(task: str, method: str, root: Path) -> pd.DataFrame:
    """Load baseline CSV for a task.
    
    Args:
        task: Normalized task name (e.g., 'cup-catch', 'h1-slide-v0').
        method: Baseline method name.
        root: Path to baseline CSV directory.
    
    Returns:
        DataFrame with columns [step, reward, seed, task, method], or empty if not found.
    """
    # For BMPC_humanoid, use BMPC task name mapping and label as "BMPC"
    if method == "BMPC_humanoid":
        csv_task = BMPC_TASK_MAP.get(task, task)
        display_method = "BMPC"
    else:
        csv_task = task
        display_method = method
    
    csv_path = root / f"{csv_task}.csv"
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        df["task"] = task  # Use normalized task name
        df["method"] = display_method
        return df
    return pd.DataFrame()


def load_all_baselines(tasks: list, max_step: int) -> pd.DataFrame:
    """Load all baseline results for given tasks.
    
    Args:
        tasks: List of normalized task names.
        max_step: Maximum step to include.
    
    Returns:
        Combined DataFrame of all baselines.
    """
    frames = []
    missing = {method: [] for method in BASELINE_ROOTS}
    
    for method, root in BASELINE_ROOTS.items():
        if not root.exists():
            print(f"Warning: {method} root not found: {root}")
            continue
            
        for task in tasks:
            df = load_baseline_for_task(task, method, root)
            if df.empty:
                missing[method].append(task)
            else:
                df = df[df["step"] <= max_step]
                frames.append(df)
    
    # Report missing (combine BMPC and BMPC_humanoid for cleaner output)
    for method, tasks_missing in missing.items():
        if method == "BMPC_humanoid":
            continue  # Skip, we'll report combined
        if tasks_missing:
            print(f"{method}: missing {len(tasks_missing)} tasks: {tasks_missing[:5]}..." 
                  if len(tasks_missing) > 5 else f"{method}: missing {tasks_missing}")
    
    if not frames:
        return pd.DataFrame()
    
    # Combine and deduplicate (in case both BMPC and BMPC_humanoid loaded same task)
    result = pd.concat(frames, ignore_index=True)
    result = result.drop_duplicates(subset=["method", "task", "step", "seed"])
    return result

In [69]:
# Get unique tasks from our results
our_tasks = sorted(ours_df["task"].unique().tolist())
print(f"Our tasks ({len(our_tasks)}): {our_tasks}")

Our tasks (5): ['dog-run', 'dog-walk', 'h1-slide-v0', 'h1-walk-v0', 'humanoid-walk']


In [70]:
# Load baselines
baselines_df = load_all_baselines(our_tasks, MAX_STEP)
print(f"\nBaseline results: {len(baselines_df)} rows")

if not baselines_df.empty:
    coverage = baselines_df.groupby('method')['task'].nunique()
    print(f"\nTask coverage per method:")
    for method, count in coverage.items():
        print(f"  {method}: {count}/{len(our_tasks)} tasks")

TDMPC2: missing ['h1-slide-v0', 'h1-walk-v0']
DreamerV3: missing ['h1-slide-v0', 'h1-walk-v0']
SAC: missing ['h1-slide-v0', 'h1-walk-v0']
EZ2: missing ['dog-run', 'dog-walk', 'h1-slide-v0', 'h1-walk-v0', 'humanoid-walk']
BMPC: missing ['h1-slide-v0', 'h1-walk-v0']

Baseline results: 1075 rows

Task coverage per method:
  BMPC: 5/5 tasks
  DreamerV3: 3/5 tasks
  SAC: 3/5 tasks
  SimbaV2: 5/5 tasks
  TDMPC2: 3/5 tasks


## Combine Data

In [71]:
# Add method column to our results
ours_df["method"] = "Ours"

# Filter our results to max step
ours_filtered = ours_df[ours_df["step"] <= MAX_STEP].copy()

# Combine all data
all_data = pd.concat([ours_filtered, baselines_df], ignore_index=True)
print(f"Total rows: {len(all_data)}")
print(f"Methods: {sorted(all_data['method'].unique().tolist())}")

Total rows: 1173
Methods: ['BMPC', 'DreamerV3', 'Ours', 'SAC', 'SimbaV2', 'TDMPC2']


## Plotting Functions

In [72]:
def hex_to_rgba(hex_color: str, alpha: float) -> str:
    """Convert hex color to rgba string."""
    hex_color = hex_color.lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f"rgba({r},{g},{b},{alpha})"


def compute_stats(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
    """Compute mean and std across seeds."""
    return df.groupby(group_cols).agg(
        mean_reward=("reward", "mean"),
        std_reward=("reward", "std"),
        n_seeds=("reward", "count")
    ).reset_index()

In [73]:
def plot_task_comparison(
    data: pd.DataFrame,
    task: str,
    colors: dict,
) -> go.Figure:
    """Create a comparison plot for a single task.
    
    Args:
        data: DataFrame with columns [step, reward, method, task].
        task: Task name to plot.
        colors: Dict mapping method names to hex colors.
    
    Returns:
        Plotly Figure, or None if no data for task.
    """
    task_data = data[data["task"] == task]
    if task_data.empty:
        return None
    
    stats = compute_stats(task_data, ["method", "step"])
    fig = go.Figure()
    
    # Plot Ours first, then baselines
    methods = ["Ours"] + [m for m in sorted(stats["method"].unique()) if m != "Ours"]
    
    for method in methods:
        method_stats = stats[stats["method"] == method].sort_values("step")
        if method_stats.empty:
            continue
        
        color = colors.get(method, "#666666")
        x = method_stats["step"].values
        y_mean = method_stats["mean_reward"].values
        y_std = np.nan_to_num(method_stats["std_reward"].values, nan=0)
        
        # Shaded std region - use numpy arrays
        x_fill = np.concatenate([x, x[::-1]])
        y_fill = np.concatenate([y_mean + y_std, (y_mean - y_std)[::-1]])
        
        fig.add_trace(go.Scatter(
            x=x_fill,
            y=y_fill,
            fill="toself",
            fillcolor=hex_to_rgba(color, 0.2),
            line=dict(color="rgba(0,0,0,0)"),
            showlegend=False,
            hoverinfo="skip",
        ))
        
        # Mean line (solid for Ours, dashed for baselines)
        line_style = dict(color=color, width=2)
        if method != "Ours":
            line_style["dash"] = "dash"
        
        fig.add_trace(go.Scatter(
            x=x,
            y=y_mean,
            mode="lines",
            name=method,
            line=line_style,
            hovertemplate=f"{method}<br>Step: %{{x:,}}<br>Reward: %{{y:.1f}}<extra></extra>",
        ))
    
    fig.update_layout(
        title=dict(text=f"<b>{task}</b>", x=0.5),
        xaxis_title="Environment Steps",
        yaxis_title="Episode Reward",
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
        hovermode="x unified",
        template="plotly_white",
        width=700,
        height=450,
    )
    
    return fig

## Individual Task Plots

In [74]:
# Generate plots for all tasks
figures = {}
for task in our_tasks:
    fig = plot_task_comparison(all_data, task, COLORS)
    if fig is not None:
        figures[task] = fig

print(f"Generated {len(figures)} task plots")

Generated 5 task plots


In [75]:
# Show first 4 plots
for task in list(figures.keys())[:4]:
    print(f"\n--- {task} ---")
    figures[task].show()


--- dog-run ---



--- dog-walk ---



--- h1-slide-v0 ---



--- h1-walk-v0 ---


## Grid View (All Tasks)

In [76]:
def create_grid_figure(
    data: pd.DataFrame,
    tasks: list,
    colors: dict,
    cols: int = 3,
) -> go.Figure:
    """Create grid of comparison plots for all tasks.
    
    Args:
        data: Combined DataFrame with all methods.
        tasks: List of task names to plot.
        colors: Dict mapping method names to hex colors.
        cols: Number of columns in grid.
    
    Returns:
        Plotly Figure with subplots.
    """
    n_tasks = len(tasks)
    rows = (n_tasks + cols - 1) // cols
    
    fig = make_subplots(
        rows=rows,
        cols=cols,
        subplot_titles=tasks,
        vertical_spacing=0.08,
        horizontal_spacing=0.05,
    )
    
    legend_added = set()
    
    for idx, task in enumerate(tasks):
        row = idx // cols + 1
        col = idx % cols + 1
        
        task_data = data[data["task"] == task]
        if task_data.empty:
            continue
        
        stats = compute_stats(task_data, ["method", "step"])
        methods = ["Ours"] + [m for m in sorted(stats["method"].unique()) if m != "Ours"]
        
        for method in methods:
            method_stats = stats[stats["method"] == method].sort_values("step")
            if method_stats.empty:
                continue
            
            color = colors.get(method, "#666666")
            x = method_stats["step"].values
            y_mean = method_stats["mean_reward"].values
            y_std = np.nan_to_num(method_stats["std_reward"].values, nan=0)
            
            show_legend = method not in legend_added
            legend_added.add(method)
            
            # Shaded region - use numpy arrays
            x_fill = np.concatenate([x, x[::-1]])
            y_fill = np.concatenate([y_mean + y_std, (y_mean - y_std)[::-1]])
            
            fig.add_trace(
                go.Scatter(
                    x=x_fill,
                    y=y_fill,
                    fill="toself",
                    fillcolor=hex_to_rgba(color, 0.15),
                    line=dict(color="rgba(0,0,0,0)"),
                    showlegend=False,
                    hoverinfo="skip",
                    legendgroup=method,
                ),
                row=row, col=col
            )
            
            # Mean line
            line_style = dict(color=color, width=1.5)
            if method != "Ours":
                line_style["dash"] = "dash"
            
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=y_mean,
                    mode="lines",
                    name=method,
                    line=line_style,
                    showlegend=show_legend,
                    legendgroup=method,
                    hovertemplate=f"{method}: %{{y:.0f}}<extra></extra>",
                ),
                row=row, col=col
            )
    
    fig.update_layout(
        height=280 * rows,
        width=1000,
        title_text="<b>DM-Control Benchmark (2 Enc Layers): Ours vs Baselines</b>",
        title_x=0.5,
        showlegend=True,
        legend=dict(x=1.01, y=1, bgcolor="rgba(255,255,255,0.9)"),
        template="plotly_white",
    )
    
    return fig

In [77]:
grid_fig = create_grid_figure(all_data, our_tasks, COLORS, cols=3)
grid_fig.show()

## Normalized Aggregate Performance

Average performance across all tasks, normalized by max performance per task.

In [78]:
def compute_normalized_aggregate(
    data: pd.DataFrame,
    eval_step: int = 100_000,
    common_steps: list = None,
) -> pd.DataFrame:
    """Compute normalized aggregate performance across tasks.
    
    For each task, normalize rewards by max performance at eval_step.
    Interpolate to common steps, then average across tasks per method.
    
    Args:
        data: Combined DataFrame with [step, reward, method, task].
        eval_step: Step at which to compute max for normalization.
        common_steps: List of steps to interpolate to. If None, uses [0, 10k, ..., eval_step].
    
    Returns:
        DataFrame with [method, step, mean_normalized, std_normalized, n_tasks].
    """
    if common_steps is None:
        common_steps = list(range(0, eval_step + 1, 10_000))
    
    # First, compute max reward per task at eval_step (across all methods)
    task_max = {}
    for task in data["task"].unique():
        task_data = data[data["task"] == task]
        max_reward = 0
        for method in task_data["method"].unique():
            method_data = task_data[task_data["method"] == method]
            available_steps = method_data["step"].unique()
            valid_steps = available_steps[available_steps <= eval_step]
            if len(valid_steps) > 0:
                step = valid_steps.max()
                rewards_at_step = method_data[method_data["step"] == step]["reward"]
                max_reward = max(max_reward, rewards_at_step.mean())
        task_max[task] = max_reward if max_reward > 0 else 1
    
    print("Max rewards per task (for normalization):")
    for task, max_r in sorted(task_max.items()):
        print(f"  {task}: {max_r:.1f}")
    
    # Normalize all rewards
    data_normalized = data.copy()
    data_normalized["normalized_reward"] = data_normalized.apply(
        lambda r: r["reward"] / task_max[r["task"]], axis=1
    )
    
    # Interpolate each (method, task) to common steps
    interpolated_rows = []
    for (method, task), group in data_normalized.groupby(["method", "task"]):
        # Average across seeds first, then interpolate
        seed_avg = group.groupby("step")["normalized_reward"].mean().reset_index()
        seed_avg = seed_avg.sort_values("step")
        
        # Interpolate to common steps
        for target_step in common_steps:
            if target_step < seed_avg["step"].min():
                val = seed_avg["normalized_reward"].iloc[0] if len(seed_avg) > 0 else 0
            elif target_step > seed_avg["step"].max():
                val = seed_avg["normalized_reward"].iloc[-1]
            else:
                val = np.interp(target_step, seed_avg["step"], seed_avg["normalized_reward"])
            
            interpolated_rows.append({
                "method": method,
                "task": task,
                "step": target_step,
                "normalized_reward": val,
            })
    
    interpolated_df = pd.DataFrame(interpolated_rows)
    
    # Aggregate across tasks
    aggregate = interpolated_df.groupby(["method", "step"]).agg(
        mean_normalized=("normalized_reward", "mean"),
        std_normalized=("normalized_reward", "std"),
        n_tasks=("normalized_reward", "count")
    ).reset_index()
    
    return aggregate

In [79]:
def plot_normalized_aggregate(
    aggregate_df: pd.DataFrame,
    colors: dict,
) -> go.Figure:
    """Plot normalized aggregate performance across tasks.
    
    Args:
        aggregate_df: DataFrame from compute_normalized_aggregate.
        colors: Dict mapping method names to hex colors.
    
    Returns:
        Plotly Figure.
    """
    fig = go.Figure()
    
    # Plot Ours first, then baselines
    methods = ["Ours"] + [m for m in sorted(aggregate_df["method"].unique()) if m != "Ours"]
    
    for method in methods:
        method_data = aggregate_df[aggregate_df["method"] == method].sort_values("step")
        if method_data.empty:
            continue
        
        color = colors.get(method, "#666666")
        x = method_data["step"].values
        y_mean = method_data["mean_normalized"].values
        y_std = np.nan_to_num(method_data["std_normalized"].values, nan=0)
        
        # Shaded std region - use numpy arrays
        x_fill = np.concatenate([x, x[::-1]])
        y_fill = np.concatenate([y_mean + y_std, (y_mean - y_std)[::-1]])
        
        fig.add_trace(go.Scatter(
            x=x_fill,
            y=y_fill,
            fill="toself",
            fillcolor=hex_to_rgba(color, 0.2),
            line=dict(color="rgba(0,0,0,0)"),
            showlegend=False,
            hoverinfo="skip",
        ))
        
        # Mean line
        line_style = dict(color=color, width=2.5)
        if method != "Ours":
            line_style["dash"] = "dash"
            line_style["width"] = 2
        
        fig.add_trace(go.Scatter(
            x=x,
            y=y_mean,
            mode="lines",
            name=method,
            line=line_style,
            hovertemplate=f"{method}<br>Step: %{{x:,}}<br>Normalized: %{{y:.3f}}<extra></extra>",
        ))
    
    fig.update_layout(
        title=dict(text="<b>Normalized Aggregate Performance (Mean over Tasks)</b>", x=0.5),
        xaxis_title="Environment Steps",
        yaxis_title="Normalized Reward (fraction of max)",
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
        hovermode="x unified",
        template="plotly_white",
        width=800,
        height=500,
        yaxis=dict(range=[0, 1.05]),
    )
    
    return fig

In [80]:
# Compute and plot normalized aggregate
aggregate_df = compute_normalized_aggregate(all_data, eval_step=MAX_STEP)
aggregate_fig = plot_normalized_aggregate(aggregate_df, COLORS)
aggregate_fig.show()

Max rewards per task (for normalization):
  dog-run: 552.1
  dog-walk: 961.1
  h1-slide-v0: 532.6
  h1-walk-v0: 844.7
  humanoid-walk: 917.4


## Summary Table

In [81]:
def create_summary_table(data: pd.DataFrame, eval_step: int = None) -> pd.DataFrame:
    """Create summary table with final mean/std per method per task.
    
    Args:
        data: Combined DataFrame.
        eval_step: Step at which to evaluate. If None, uses max step per method.
    
    Returns:
        DataFrame with one row per task, columns for each method's mean/std.
    """
    results = []
    methods = sorted(data["method"].unique())
    
    for task in sorted(data["task"].unique()):
        task_data = data[data["task"] == task]
        row = {"task": task}
        
        for method in methods:
            method_data = task_data[task_data["method"] == method]
            if method_data.empty:
                row[f"{method}_mean"] = np.nan
                row[f"{method}_std"] = np.nan
                continue
            
            # Get data at eval step or max available
            if eval_step is not None:
                available_steps = method_data["step"].unique()
                valid_steps = available_steps[available_steps <= eval_step]
                if len(valid_steps) == 0:
                    step = available_steps.max()
                else:
                    step = valid_steps.max()
            else:
                step = method_data["step"].max()
            
            final_data = method_data[method_data["step"] == step]
            row[f"{method}_mean"] = final_data["reward"].mean()
            row[f"{method}_std"] = final_data["reward"].std()
        
        results.append(row)
    
    return pd.DataFrame(results)

In [82]:
# Summary at 100k steps (this sweep's max)
summary_100k = create_summary_table(all_data, eval_step=100_000)
print("Summary at 100k steps (or max available):")
summary_100k.round(1)

Summary at 100k steps (or max available):


,task,BMPC_mean,BMPC_std,DreamerV3_mean,DreamerV3_std,Ours_mean,Ours_std,SAC_mean,SAC_std,SimbaV2_mean,SimbaV2_std,TDMPC2_mean,TDMPC2_std
0,dog-run,35.1,23.8,0.0,NaN,148.5,15.1,8.3,2.5,106.2,9.9,26.6,16.7
1,dog-walk,67.9,16.0,9.0,NaN,377.0,12.6,12.6,3.3,231.7,51.0,34.6,14.8
2,h1-slide-v0,24.8,NaN,NaN,NaN,58.3,NaN,NaN,NaN,39.4,4.1,NaN,NaN
3,h1-walk-v0,9.4,NaN,NaN,NaN,9.4,NaN,NaN,NaN,45.6,7.2,NaN,NaN
4,humanoid-walk,1.1,0.1,0.6,NaN,2.1,0.5,1.1,0.3,31.5,65.8,1.5,0.1


In [83]:
def format_summary_pretty(df: pd.DataFrame) -> pd.DataFrame:
    """Format summary with 'mean ± std' columns."""
    methods = sorted(set(col.replace("_mean", "") for col in df.columns if col.endswith("_mean")))
    
    formatted = df[["task"]].copy()
    for method in methods:
        mean_col = f"{method}_mean"
        std_col = f"{method}_std"
        if mean_col in df.columns:
            formatted[method] = df.apply(
                lambda r: f"{r[mean_col]:.0f} ± {r[std_col]:.0f}" 
                if pd.notna(r[mean_col]) else "—",
                axis=1
            )
    return formatted

pretty_summary = format_summary_pretty(summary_100k)
pretty_summary

,task,BMPC,DreamerV3,Ours,SAC,SimbaV2,TDMPC2
0,dog-run,35 ± 24,0 ± nan,148 ± 15,8 ± 2,106 ± 10,27 ± 17
1,dog-walk,68 ± 16,9 ± nan,377 ± 13,13 ± 3,232 ± 51,35 ± 15
2,h1-slide-v0,25 ± nan,—,58 ± nan,—,39 ± 4,—
3,h1-walk-v0,9 ± nan,—,9 ± nan,—,46 ± 7,—
4,humanoid-walk,1 ± 0,1 ± nan,2 ± 1,1 ± 0,32 ± 66,1 ± 0


## Save Results

In [84]:
# Create output directory
output_dir = paths.notebook_results_dir("11_dmcontrol_2enc_benchmark")
print(f"Output directory: {output_dir}")

# Save summary tables
summary_100k.to_csv(output_dir / "summary_100k.csv", index=False)
pretty_summary.to_csv(output_dir / "summary_100k_pretty.csv", index=False)

# Save grid figure
grid_fig.write_html(output_dir / "grid_comparison.html")

# Save normalized aggregate results
aggregate_df.to_csv(output_dir / "normalized_aggregate.csv", index=False)
aggregate_fig.write_html(output_dir / "normalized_aggregate.html")

# Save individual task figures
for task, fig in figures.items():
    safe_name = task.replace("-", "_")
    fig.write_html(output_dir / f"{safe_name}.html")

print(f"\nSaved:")
print(f"  - summary_100k.csv")
print(f"  - summary_100k_pretty.csv")
print(f"  - grid_comparison.html")
print(f"  - normalized_aggregate.csv")
print(f"  - normalized_aggregate.html")
print(f"  - {len(figures)} individual task plots")

Output directory: /gpfs/work4/0/prjs0951/Thomas/Thesis/RL_weather/tdmpc2-with-return-based-auxiliary-tasks.worktrees/main/analysis/results/11_dmcontrol_2enc_benchmark

Saved:
  - summary_100k.csv
  - summary_100k_pretty.csv
  - grid_comparison.html
  - normalized_aggregate.csv
  - normalized_aggregate.html
  - 5 individual task plots
